# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install async module
!uv pip install nest_asyncio

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# Import client
from syft_nsai_sdk import Client
from syft_nsai_sdk.utils.logger import get_logger

In [4]:
# define utilities
nest_asyncio.apply()
logger = get_logger()
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)
logger.success("Logger initialized successfully!")

2025-09-04 12:12:29 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# Initialize client
client = Client()

2025-09-04 12:12:30 - syft_nsai_sdk.main - INFO - Found existing accounting credentials for callis@openmined.org
2025-09-04 12:12:30 - syft_nsai_sdk.main - INFO - Client initialized for callis@openmined.org


## Basic usage

In [10]:
# inline service usage - , "bob@example.com/wiki"
inline_result = client.pipeline(
    data_sources=["callis@openmined.org/sota-free", "callis@openmined.org/carl-model"], 
    synthesizer="callis@openmined.org/sota-free"
).run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             



In [11]:
# show response
logger.info(f"Pipeline result:\n\n {inline_result}")

2025-09-04 11:57:27 - syft_nsai_sdk - INFO - Pipeline result:

 Snippet 1. Code Interpretation Extension can generate and run Python code:

The Snippet describes how the Extension can interpret and run Python code. The selection and invocation of the Extension is guided by the use of Example, which are defined as part of the Extension configuration.


In [9]:
# Object-Oriented service usage
pipeline = client.create_pipeline()
pipeline.add_source("callis@openmined.org/sota-free", topK=8)
pipeline.add_source("callis@openmined.org/carl-model", topK=8)
pipeline.set_synthesizer("callis@openmined.org/sota-free", temperature=0.7)  
oop_result = pipeline.run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             

✓ Response received                                                             



In [10]:
# show response
logger.info(f"Pipeline result:\n\n {oop_result}")

2025-09-04 12:01:38 - syft_nsai_sdk - INFO - Pipeline result:

 An agent is a self-directed artificial intelligence (AI) program capable of reasoning, logic, and accessing external information. This concept is based on the integration of different tools that include user interfaces, evaluation frameworks, and continuous improvement mechanisms. The software architecture also offers a set of development tools that allow developers to focus on building and refining their agents while managing the complexities of infrastructure, deployment, and maintenance. An example of this is Vertex AI platform, which simplifies the process of defining crucial elements like goals, task instructions, tools, sub-agents for task delegation, and examples - allowing developers to focus on building and refining their agents while managing the complexities of infrastructure, deployment, and maintenance.


## Advanced usage

In [12]:
# Load some services as objects first
carl_model = client.load("callis@openmined.org/carl-model") # paid
sota_free = client.load("callis@openmined.org/sota-free") # free
carl_free = client.load("callis@openmined.org/carl-free") # free
carl_claude = client.load("callis@openmined.org/carl-claude") # free

In [13]:
# Demo mixed input types
mixed_result = client.pipeline_mixed(
    data_sources=[
        "callis@openmined.org/sota-free",           # String
        carl_free,                                       # Service object
        {"name": "callis@openmined.org/carl-claude", "topK": 5},  # Dict with string
        {"name": sota_free, "topK": 8}                   # Dict with Service object
    ],
    synthesizer=docs  # Service object as synthesizer
).run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             

✓ Response received                                                             

                                                                                

2025-09-04 13:13:25 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

                                                                                

2025-09-04 13:13:26 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-04 13:13:26 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-04 13:13:26 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             



In [7]:
logger.info(f"Mixed pipeline result:\n\n {mixed_result}")

2025-09-04 12:14:53 - syft_nsai_sdk - INFO - Mixed pipeline result:

 An agent is a self-directed program or system that can plan and execute tasks in a specific context. It includes reasoning, logic, access to external information, and ability to plan and execute tasks beyond the capabilities of a single AI model. In this context, an example is a vertix ai platform that provides developers with a set of tools for building and refining their agents while managing complexities of infrastructure, deployment, and maintenance by itself. The whitepaper dives into all these aspects in more detail, including the sample architecture of an agent that generates code from a given query.


In [8]:
# Demo with pure Service objects
service_result = client.pipeline_mixed(
    data_sources=[sota_free, carl_free],
    synthesizer=sota_free
).run(messages=[{"role": "user", "content": "What is machine learning?"}])

2025-09-04 12:17:03 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-04 12:17:03 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             



In [9]:
logger.info(f"Service objects pipeline result:\n\n {service_result}")

2025-09-04 12:17:30 - syft_nsai_sdk - INFO - Service objects pipeline result:

 Machine learning (ML) is a type of AI that uses algorithms to learn from and improve upon data without being explicitly programmed. In ML, data is used to train models that can make predictions or recommendations based on their knowledge of the data. This process is often automated, meaning algorithms can continuously analyze new data and improve their predictions over time. 

In general, ML can be broken down into three main components: training data, model, and algorithm. The training data (also known as the "training set") contains the original data used to train the model. This data is typically labeled or tagged with relevant information such as categories or attributes. Once the training set is complete, the model can be created by using algorithms that fit the training data.

During the modeling phase, algorithms will use a variety of techniques to analyze the data and develop predictions based on th

In [10]:
# Demo Service objects with different parameters
param_result = client.pipeline_mixed(
    data_sources=[
        {"name": sota_free, "topK": 10},
        {"name": "callis@openmined.org/carl-claude", "topK": 3}
    ],
    synthesizer={"name": sota_free, "temperature": 0.9}
).run(messages=[{"role": "user", "content": "Explain neural networks"}])

2025-09-04 12:19:40 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-04 12:19:40 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-model: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             



In [11]:
logger.info(f"Parameters pipeline result:\n\n {param_result}")

2025-09-04 12:20:34 - syft_nsai_sdk - INFO - Parameters pipeline result:

 Newwhitepaper_Agent2.pdf
The goal of an agent in AI is not just to act as a "human" in the context of explainable artificial intelligence (XAI) models, but also to have a clear understanding and reasoning about the inner working of such models. An agent's cognitive architecture, or how it operates, consists of three essential components: combinational reasoning, logic, and access to external information. These components help agents process information, make decisions, and take actions.

In this whitepaper, we will introduce these components and the different types of agents that Generative AI models can build at the time of publication. In order to understand the inner working of an agent, let's first introduce the foundational components that drive the agent's behavior, actions, and decision-making.

Combinational Reasoning: This is combination of reasoning, logic, and access to external information that are a